# <h2 align=center> PROYECTO FINAL </h2>
# <h1 align=center>YELP & GOOGLE MAPS - REVIEWS AND RECOMMENDATIONS</h1>
        Proyecto Final Grupo Nro. 7
        @utores:  Javier Castro, Luca Ramallo, Luis Ramirez, Lesmen Garcia.

#### Objetivo:<br> Realizar un análisis de la información de reseñas entre las plataformas de Yelp y Google Maps en los últimos 5 años, para identificar oportunidades de inversión.
### Objetivo Específicos:<Br>
1. Seleccionar las oportunidades de inversión de las reseñas de informacion que se encuentren correlacionadas entre ambas plataformas.<br>
2. Determinar la satisfacción, percepción y tendencia de los clientes de acuerdo a las reseñas realizadas.<br> 
3. Evaluar la popularidad y crecimiento de los diferentes tipos de negocios de acuerdo al historial de reseñas.<br>
4. Determinar competencias entre negocios del mismo ramo, por ubicación greográfica.<br>
5. Evaluar y clasificar la calidad del servicio al cliente ofrecido por cada tipo de negocio.<br>
6. Determinar el rendimiento financiero por negocio en el periodo de tiempo establecido.<br>

### Versiones de las librerias utilizadas<Br>
+   pandas==1.3.5<Br>
+   numpy==1.24.3<Br>
+   pip==23.2.1<Br>
+   scikit-learn==1.0.2<Br>
+   regex==2023.5.5<Br>
+   matplotlib==3.7.1<Br>
+   seaborn=0.12.2<Br>
+   requests==2.31.0<Br>

Importando Librerias

In [248]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import json
import requests
from datetime import datetime, timedelta
import random
from scipy.stats import norm


# 2. Analisis Exploratorios de los Datos (EDA)